# TFX Interactive Development

In [1]:
import absl
import os
import tempfile

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.components import CsvExampleGen
from tfx.components import BigQueryExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input


print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)

Tensorflow Version: 1.15.0
TFX Version: 0.15.0


## Configure lab settings

Set location paths, connections strings, and other environment settings. Make sure to update project id, region, and artifact store GCS bucket with your settings.

In [2]:
PROJECT_ID = 'mlops-workshop'
ARTIFACT_STORE='gs://mlops-workshop-artifact-store'
REGION = 'us-central1'

PIPELINE_NAME='tfx-covertype-classifier'
PIPELINE_ROOT='{}/{}'.format('/home/tfx-interactive', PIPELINE_NAME)
os.makedirs(PIPELINE_ROOT, exist_ok=True)

DATA_ROOT = 'gs://workshop-datasets/covertype/'
TRAINING_FILES = 'training/*'
EVALUATION_FILES = 'evaluation_anomalies/*'

## TFX Component Walk-through

### Create Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional metadata_store_pb2.ConnectionConfig instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [3]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None
)

### Ingest data using ExampleGen

The ExampleGen TFX Pipeline component ingests data into TFX pipelines. It consumes external files/services to generate Examples which will be read by other TFX components. It can also split and shuffle the data.

- Consumes: Data from external data sources such as CSV, TFRecord and BigQuery
- Emits: tf.Example records

#### Configure CsvExampleGen


In [4]:
input_config = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern=TRAINING_FILES),
                example_gen_pb2.Input.Split(name='eval', pattern=EVALUATION_FILES)
            ])

example_gen = tfx.components.CsvExampleGen(
    instance_name='Data_Extraction_Spliting',
    input=external_input(DATA_ROOT),
    input_config=input_config
)

#### Run ExampleGen

In [5]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen.Data_Extraction_Spliting
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/CsvExampleGen.Data_Extraction_Spliting/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/CsvExampleGen.Data_Extraction_Spliting/examples/1/eval/, split: eval, id: 3)]
        ))

### Generate statistics using StatisticsGen

The StatisticsGen  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: datasets created by ExampleGen 
- Emits: Dataset statistics

#### Run StatisticsGen

In [6]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx-interactive/tfx-covertype-classifier/StatisticsGen.Statistics_Generation/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx-interactive/tfx-covertype-classifier/StatisticsGen.Statistics_Generation/output/2/eval/, split: eval, id: 5)]
        ))

#### Display statistics

In [8]:
context.show(statistics_gen.outputs['statistics'])

### Infer data schema using SchemaGen

Some TFX components use a description of your input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: statistics from `StatisticsGen`
- Emits: Data schema proto

`SchemaGen` generates the schema using the statistics for the `train` split. Statistics for other splits are ignored.

#### Run SchemaGen

In [9]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/tfx-interactive/tfx-covertype-classifier/SchemaGen/output/3/, split: , id: 6)]
        ))

#### Visualize the inferred schema

In [10]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,-
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Elevation',INT,required,single,-
'Aspect',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Cover_Type',INT,required,single,-


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"


### Validate data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema. 

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


- Consumes: A schema 
- Emits: Validation results

`ExampleValidator` validates the data in the `eval` split only. Other splits are ignored. 

#### Run ExampleValidator


In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['output'],
    instance_name="Data_Validation"
)

context.run(example_validator, enable_cache=False)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 4
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /home/tfx-interactive/tfx-covertype-classifier/ExampleValidator.Data_Validation/output/4/, split: , id: 7)]
        ))

#### Visualize validation results

In [14]:
context.show(example_validator.outputs['output'])

,Anomaly short description,Anomaly long description
Feature name,,
'Soil_Type',Unexpected string values,Examples contain values missing from the schema: C5151 (<1%).


As noted, the `SchemaGen` component generates the schema from the `train` split. The `Soil_Type` feature in the `train` fit does not include the `C5151` value and that is why the `eval` split fails validation. 

In most cases the auto-generated schemas must be fine tuned manually using insights from data exploration and/or domain knowledge about the data.

#### Update the schema

Load the schema from the schema proto file

In [16]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

Add the `C5151` value to the domain of `Soil_Type` and define the domain of `Cover_Type` as the 1-7 range - there are 7 types of forest cover encoded as integers from 1 to 7.

In [17]:
tfdv.get_domain(schema, 'Soil_Type').value.append('C5151')
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=1, max=7, is_categorical=True))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,-
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Elevation',INT,required,single,-
'Aspect',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Cover_Type',INT,required,single,"[1,7]"


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"


Save the updated schema to a proto file.

In [18]:
schema_dir = '/home/covertype_schema'
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Horizontal_Distance_To_Roadways"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Slope"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Wilderness_Area"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "Wilderness_Area"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_Noon"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  pr

### Import the updated schema using ImporterNode

The `ImporterNode` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


#### Import the schema file

In [19]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 5
    outputs:
        result: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/covertype_schema, split: , id: 8)]
        ))

#### Visualize the imported schema

In [20]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,-
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Elevation',INT,required,single,-
'Aspect',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Cover_Type',INT,required,single,"[1,7]"


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"


#### Validate data using the updated schema


In [21]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
    instance_name="Data_Validation"
)

context.run(example_validator, enable_cache=False)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 6
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /home/tfx-interactive/tfx-covertype-classifier/ExampleValidator.Data_Validation/output/6/, split: , id: 9)]
        ))

In [22]:
context.show(example_validator.outputs['output'])